# Forcing Extraction Tutorial

In this notebook, we'll present an example of converting PRISM precipitation, mean temperature, and mean dew point data into a format that can be used easily in the types of vadose and groundwater zone models we cover in the [custom zone models tutorial](custom_zone_models.ipynb). 

In [1]:
from matplotlib import pyplot as plt
import matplotlib 
%matplotlib inline
import pandas as pd
import gdal
import os
import numpy as np
import fiona
import shapely
from shapely import geometry
from os.path import dirname
import glob
import sys
import pickle
from functools import partial
from datetime import datetime
from tqdm import tqdm_notebook

parent_dir = dirname(dirname(os.getcwd()))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','lib'))
rew_config = pickle.load( open( os.path.join(parent_dir,'model_data','rew_config.p'), "rb" ) )
import zonal_stats as zs
import meteolib as meteo
import evaplib as evap

model_config = pickle.load( open( os.path.join(parent_dir, 'model_data', 'model_config.p'), 'rb'))
start_date = model_config['start_date']
stop_date = model_config['stop_date']

We first need to specify the geographic areas of interest, comprised of the collection of REW sub-basins. We'll assume that these regions are specified via shapefiles located in the `raw_data` folder. 

In [2]:
try:
    basins = glob.glob(os.path.join(parent_dir,'raw_data','basins_poly','*.shp'))[0]
except RuntimeError:
    print 'Cannot find basins shapefile. Please make sure basins shapefile is located in \n the model directory under /raw_data/basins_poly'

For simplicity, we'll assume that the forcing along the spatial extent of each basin is constant and equal to the forcing observed at its centroid. We could also consider implementing zonal averaging or some other more faithful approximation, but this would be much more computationally intensive, and quite likely unnecessary for lower resolution climate datasets. 

In [3]:
# TODO CONSIDER CHANGING THIS TO JUST READ FROM THE CENTROIDS ALREADY GENERATED
# ALSO NOTE: new projection does not return lat/long points for xy coordinates
fc = fiona.open(basins)
shapefile_record = fc.next()
pos_dict={}
for shapefile_record in fc:
    shape = shapely.geometry.asShape(shapefile_record['geometry'])
    long_point = shape.centroid.coords.xy[0][0]
    lat_point = shape.centroid.coords.xy[1][0]
    pos = (long_point, lat_point)
    pos_dict[int(shapefile_record['properties']['cat'])]=pos

With the list of centroids in hand, we can begin unpacking the forcing data. Here we'll focus on three time series: daily precipitation (`ppt`), mean daily temperature (`tmean`), and mean daily dew point (`tdmean`). 

Our main data structure here will be a dictionary whose keys are forcing data types and whose values are themselves dictionaries. These inner dictionaries have REW IDs as keys, and forcing data at particular instances of time as values.

We track time in a separate dictionary, again keyed based on forcing data. The values here are lists of dates at which each type of forcing data was observed. 

Note: we assume that each forcing time series is stored exactly as would be downloaded from the PRISM download page. A simple bash script for downloading daily PRISM data can be found [HERE](https://github.com/daviddralle/downloadPrism). The files must be unzipped after download. For the purposes of the model, the data should be stored in the `raw_data` folder in a foldername equal to the variable name, for instance precipitation (`ppt`) is in `raw_data/ppt`.

In [4]:

pos_dict = {1:(444800, 4397800), 2:(459400, 4380000)}

In [5]:
stop_date

datetime.date(2017, 8, 30)

In [6]:
forcing_dict = {}
dates_dict = {}

start_date = model_config['start_date']
stop_date = datetime.date(pd.to_datetime('2017-09-30'))

prism_vars = ['ppt','tmean', 'tmin', 'tmax']

for prism_var in prism_vars:
    print "Extracting " + str(prism_var) + " data..."
    years = os.listdir(os.path.join(parent_dir,'raw_data',prism_var))

    # Initialize empty data structures -- a dict keyed on REW ID with empty lists as values, 
    # and an empty list that will hold dates at which prism_var was observed
    vals_dict = {k:[] for k in pos_dict.keys()}
    date_list=[]
    
    #for all years of forcing variable, load each day of raster data and extract to all REWs
    for year in years:
        try: 
            int(year)
        except ValueError:
            continue

        # Assuming that data is in .tif format from PRISM.
        # Data for each REW is extracted to REW centroid lat/long. 
        raster_list = glob.glob(os.path.join(parent_dir,'raw_data',prism_var,year,'*.tif'))
        for rast in raster_list:
            datecurr = pd.to_datetime(rast[-16:][:8])
            if (datecurr.date() > stop_date)|(datecurr.date() < start_date):
                continue
            date_list.append(rast[-16:][:8])
            raster_file = os.path.join(parent_dir,'raw_data',prism_var,year,rast)
            gdata = gdal.Open(raster_file)
            gt = gdata.GetGeoTransform()
            data = gdata.ReadAsArray().astype(np.float)
            gdata = None
            for rew_id in pos_dict.keys(): 
                pos = pos_dict[rew_id]
                x = int((pos[0] - gt[0])/gt[1])
                y = int((pos[1] - gt[3])/gt[5])
                vals_dict[rew_id].append(data[y, x])
                
        print str(year) + ' ' + prism_var + ' data processed.'                

    forcing_dict[prism_var] = vals_dict
    dates_dict[prism_var] = date_list

Extracting ppt data...
2012 ppt data processed.
2013 ppt data processed.
2014 ppt data processed.
2015 ppt data processed.
2016 ppt data processed.
2017 ppt data processed.
Extracting tmean data...
2012 tmean data processed.
2013 tmean data processed.
2014 tmean data processed.
2015 tmean data processed.
2016 tmean data processed.
2017 tmean data processed.
Extracting tmin data...
2012 tmin data processed.
2013 tmin data processed.
2014 tmin data processed.
2015 tmin data processed.
2016 tmin data processed.
2017 tmin data processed.
Extracting tmax data...
2012 tmax data processed.
2013 tmax data processed.
2014 tmax data processed.
2015 tmax data processed.
2016 tmax data processed.
2017 tmax data processed.


In [7]:
ppt_sites = forcing_dict['ppt']
elder_rainfall = pd.DataFrame(forcing_dict['ppt'][1], index=pd.to_datetime(dates_dict['ppt']), columns=['ppt'])
dry_rainfall = pd.DataFrame(forcing_dict['ppt'][2], index=pd.to_datetime(dates_dict['ppt']), columns=['ppt'])
print dry_rainfall.ppt.loc['10-2014':'9-2015'].sum()
print elder_rainfall.ppt.loc['10-2014':'9-2015'].sum()

print dry_rainfall.ppt.loc['10-2016':'9-2017'].sum()
print elder_rainfall.ppt.loc['10-2016':'9-2017'].sum()

1339.65000057
1468.09000425
2866.27000164
3181.20599085


Before using the data for modeling, we need to ensure that the observation dates for each forcing time series align. We leverage Python's sets to make sure that all the datelists are in fact the same.

In [8]:
unique_dates = [list(i) for i in set(tuple(i) for i in dates_dict.values())]
if len(unique_dates)>1:
    print 'Forcing data dates do not all match, please check raw data!'
else:
    print 'Dates match!'

Dates match!


In general, we will house all timeseries data in Pandas dataframes in order to leverage some very nice resampling functionality. For now, we'll just initialize one empty dataframe for each REW. Each of these dataframes will be indexed by date and will have one column for each forcing timeseries. 

In [9]:
rng = pd.date_range(unique_dates[0][0],unique_dates[0][-1])
rew_dfs_dict = {k:pd.DataFrame(data=None,index=rng) for k in pos_dict.keys()}

We can now simply cycle over the PRISM variables we enumerated earlier. Here, we might have to convert units to be in line with the main modeling effort's assumption that lengths are in units of cm and time is in units of days. 

In [10]:
for prism_var in prism_vars:
    #need to convert ppt from mm/day to cm/day for PRISM data
    if prism_var=='ppt': div = 10.0
    else: div = 1.0
    for rew_id in pos_dict.keys():
        rew_dfs_dict[rew_id][prism_var] = np.array(forcing_dict[prism_var][rew_id])/div
        
# Remove any intercepted rainfall 
for rew_id in rew_dfs_dict.keys():
    maxvals = rew_dfs_dict[rew_id].apply(lambda row: np.max([row['ppt'] - rew_config[rew_id]['interception_factor'], 0]),axis=1)
    rew_dfs_dict[rew_id].ix[:,'ppt'] = maxvals

The last remaining forcing timeseries required by the model presented in this tutorial is daily average potential evapotranspiration (in units of cm/day). Here, we'll use the Priestly-Taylor equation to compute this forcing.   Before we get started, we need to build some helper functions. 

In [11]:
# compute relative humidity using temp and dewpoint
def get_rh(tmean, tdmean):
    a = 17.625
    b = 243.04
    return 100*np.exp(a*tdmean/(b+tdmean))/np.exp(a*tmean/(b+tmean))

def get_ea(tdmean):
    return 0.6108*np.exp((17.27*tdmean)/(tdmean+237.3))

def get_rew_pressures(rew_elevations):
    #elevations in meters, output pressure in Pa
    #using simple elevation/pressure relationship
    rew_pressures = {}
    for key in rew_elevations.keys():
        rew_pressures[key] = 1000*101.325*((293-0.0065*rew_elevations[key])/293)**5.26

    return rew_pressures

# translate feature IDs to REW IDs
def translate_to_fid(parent_dir):
    basins = glob.glob(os.path.join(parent_dir,'raw_data','basins_poly','*.shp'))[0]
    translated = {}

    fc = fiona.open(basins)
    shapefile_record = fc.next()
    for shapefile_record in fc:
        translated[int(shapefile_record['properties']['cat'])] = int(shapefile_record['id'])
    return translated

# translate REW IDs to feature IDs
def translate_to_rew_id(parent_dir):
    basins = glob.glob(os.path.join(parent_dir,'raw_data','basins_poly','*.shp'))[0]
    translated = {}

    fc = fiona.open(basins)
    shapefile_record = fc.next()
    for shapefile_record in fc:
        translated[int(shapefile_record['id'])] = int(shapefile_record['properties']['cat'])
    return translated

We finally have all the pieces to compute PET. We'll use the Priestley-Taylor implementation from the `evap` module. We assume that the soil heat flux is zero at the daily timescale. Once all forcing timeseries have been computed, we save to `rew_forcing.p` in the `model_data` directory. 

In [12]:
# Using Eqn 50 From Allen (1998) to get solar radiation from max/min temp difference
kRs = 0.18

# centroids of basins
centroids = pd.read_csv( os.path.join(parent_dir, 'raw_data','basins_centroids', 'points.csv') )

for rew_id in rew_dfs_dict.keys():
    doy = [rng[i].timetuple().tm_yday for i in range(len(rng))]
    N, Rext = meteo.sun_NR(doy, float(centroids['lat'].loc[centroids['cat']==rew_id]))
    Rext_MJ = Rext/(10.0**6)
    tmax = rew_dfs_dict[rew_id]['tmax']
    tmin = rew_dfs_dict[rew_id]['tmin']
    tmean = rew_dfs_dict[rew_id]['tmean']
    Rs = kRs*np.sqrt(tmax-tmin)*Rext
    pet = evap.hargreaves(tmin, tmax, tmean, Rext_MJ)
    pet = pet.ffill()
    rew_dfs_dict[rew_id]['pet'] = pet/10.0  #into units of cm/day

In [13]:
print('Elder PET sums')
print('For 2015 %.2f mm'%(10*rew_dfs_dict[1].pet.loc['5-2015':'9-2015'].sum()))
print('For 2017 %.2f mm'%(10*rew_dfs_dict[1].pet.loc['5-2017':'9-2017'].sum()))

print('Dry PET sums')
print('For 2015 %.2f mm'%(10*rew_dfs_dict[2].pet.loc['5-2015':'9-2015'].sum()))
print('For 2017 %.2f mm'%(10*rew_dfs_dict[2].pet.loc['5-2017':'9-2017'].sum()))

Elder PET sums
For 2015 701.08 mm
For 2017 683.71 mm
Dry PET sums
For 2015 746.48 mm
For 2017 727.36 mm


# Aggregate data to climate groups

The model itself requires forcing data to be indexed by climate groups, not REW id's. For this tutorial, each REW comprises its own climate group. However, it may be that the user would want to lump REWs with similar climates into groups, within which each REW is forced with identical data. Here, we demonstrate a straightforward aggregation procedure that would take the average of each forcing variable across the REWs in each climate group. 

In [10]:
# the forcing dictionary must have climate_group id's as keys. 
# for the time being, we will merge REWs within each climate group
# using the mean of the forcings for all REWs in the climate group. 
rew_config = pickle.load( open( os.path.join(parent_dir,'model_data','rew_config.p'), "rb" ) )
climate_group_forcing = {}
for climate_group in set([rew_config[i]['climate_group'] for i in rew_config.keys()]):
    rew_ids_in_climate_group = [i for i in rew_config.keys() if rew_config[i]['climate_group']==climate_group]
    rew_dfs  = [rew_dfs_dict[rew_id] for rew_id in rew_ids_in_climate_group]
    df = pd.concat(rew_dfs)
    df = df.groupby(df.index).mean()
    climate_group_forcing[climate_group] = df

    
# Now save the dictionary of dataframes, where the keys are climate groups and each entry is a pandas dataframe 
# containing the forcings associated with that climate group. 
pickle.dump( climate_group_forcing, open( os.path.join(parent_dir,'model_data','climate_group_forcing.p'), "wb" ) )

# Also save the original REW climatic forcing dataframe. This will be used for the temperature model
pickle.dump( rew_dfs_dict, open( os.path.join(parent_dir,'model_data','rew_forcing.p'), "wb" ) )